MNIST digits classification using 3x3 pixels basis function.


Environment

 *Google Colaboratory (GPU) with Microsoft Edge 87.0.664.47

 *Python 3.6.9

 *tensorflow 2.3.0
 
 *tensorboard 2.3.0
 
 *numpy 1.18.5



In [ ]:
# magic command in google colaboratory to Load the TensorBoard notebook extension 
%load_ext tensorboard

In [ ]:
# magic command in google colaboratory to run tensorboard
%tensorboard --logdir logs

In [ ]:
#coding:utf-8

# START MAIN PROGRAM HERE !
import tensorflow as tf
import numpy as np
import os


from tensorflow.keras.layers import Dense, Conv2D, Input, Lambda, Flatten
from tensorflow.keras.layers import Add, Reshape, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K


from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard


In [ ]:
def SabunModel(r_s3, s3=3, lout=128):
    xin = Input(shape = (s3, s3, lout)) # (s3,s3,lout)
    out = Lambda(lambda x: K.abs( x - K.reshape(r_s3,(s3,s3,1))) , output_shape=(s3,s3,lout))(xin)  # compute absolute value of difference
    out = Lambda(lambda x: K.mean(x, axis=-2,keepdims=False), output_shape=(s3,lout))(out)  # compute average step1
    out = Lambda(lambda x: K.mean(x, axis=-2,keepdims=True), output_shape=(1,lout))(out)    # compute average step2
    return  Model(inputs=[xin], outputs=out)


In [ ]:
if __name__ == '__main__':   

    nb_classes = 10  # digits numebr, that are 0,1,2,3,4,5,6,7,8,9
    
    # Load MNIST dataset
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    print("X_train original shape", X_train.shape)
    print("y_train original shape", y_train.shape)
    
    X_train = X_train.reshape(60000, 28,28,1) # train dataset (60000 samples, 28ｘ28 pixels, color 1)
    X_test = X_test.reshape(10000, 28,28,1)   # validation dataset (10000 samples, 28ｘ28 pixels, color 1)
    #  transform from grayscale integer value to float 0.0～1.0
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    print("Training matrix shape", X_train.shape)
    print("Testing matrix shape", X_test.shape)
    
    # transform from label integer to one-hot vector ex: [0,0,1,0,0]
    Y_train = to_categorical(y_train, nb_classes)
    Y_test =  to_categorical(y_test, nb_classes) 

    
    
    # eleven 3x3 pixels basis functions definition
    s3=3  # size of pixel
    # 1st
    r1_s3 = tf.constant([[0, 1, 0],  \
                         [0, 1, 0],  \
                         [0, 1, 0]], dtype='float32')
                         
    # 2nd
    r2_s3 = tf.constant([[0, 0, 1],  \
                         [0, 0, 1],  \
                         [0, 0, 1]], dtype='float32')
                         
    # 3rd
    r3_s3 = tf.constant([[1, 0, 0],  \
                         [1, 0, 0],  \
                         [1, 0, 0]], dtype='float32')
                         
    # 4th
    r4_s3 = tf.constant([[0, 0, 0],  \
                         [0, 0, 0],  \
                         [1, 1, 1]], dtype='float32')
                         
    # 5th
    r5_s3 = tf.constant([[0, 0, 0],  \
                         [1, 1, 1],  \
                         [0, 0, 0]], dtype='float32')
                         
    # 6yh
    r6_s3 = tf.constant([[1, 1, 1],  \
                         [0, 0, 0],  \
                         [0, 0, 0]], dtype='float32')
                         
    # 7th
    r7_s3 = tf.constant([[1, 1, 1],  \
                         [0, 0, 1],  \
                         [1, 1, 1]], dtype='float32')
                         
    # 8th
    r8_s3 = tf.constant([[1, 1, 1],  \
                         [1, 0, 0],  \
                         [1, 1, 1]], dtype='float32')
                         
    # 9th
    r9_s3 = tf.constant([[1, 1, 1],  \
                         [1, 0, 1],  \
                         [1, 1, 1]], dtype='float32')
                         
    # 10th
    r10_s3 = tf.constant([[0, 0, 1],  \
                         [0, 1, 0],  \
                         [1, 0, 0]], dtype='float32')
                         
    # 11th
    r11_s3 = tf.constant([[1, 0, 0],  \
                         [0, 1, 0],  \
                         [0, 0, 1]], dtype='float32')
    
    
    #　number of output layer
    lout=128
    # input image 28x28 pixels 1 color 
    inp = Input(shape = (28,28,1))
    # downsize input image from 28x28 pixels to 3x3 pixels using CNN and MaxPooling
    x = Conv2D(32,(2,2),activation='relu', padding='same')(inp)  # (28,28,32)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='valid' )(x)   # (14,14,32)
    x = Conv2D(lout,(2,2),activation='relu')(x)                              # (13,13,128)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='valid')(x)    # (6,6,128)
    x = Conv2D(lout,(2,2),activation='relu')(x)                              # (5,5,128)  
    x = MaxPooling2D(pool_size=(2, 2), strides=(1,1), padding='valid')(x)    # (4,4,128)
    x3 = Conv2D(lout,(2,2),activation='sigmoid')(x)            # (3,3,128)   # use sigmoid to output range in 0-1.0
      
    if True:
        # compute absolute value of difference from 3x3 pixels basis function
        sabun1= SabunModel(r1_s3, s3=s3, lout=lout)
        x31 = sabun1([x3])  # (1,128)
        
        sabun2= SabunModel(r2_s3, s3=s3, lout=lout)
        x32 = sabun2([x3]) 
        
        sabun3= SabunModel(r3_s3, s3=s3, lout=lout)
        x33 = sabun3([x3]) 
        
        sabun4= SabunModel(r4_s3, s3=s3, lout=lout)
        x34 = sabun4([x3]) 
        
        sabun5= SabunModel(r5_s3, s3=s3, lout=lout)
        x35 = sabun5([x3]) 
        
        sabun6= SabunModel(r6_s3, s3=s3, lout=lout)
        x36 = sabun6([x3]) 
        
        sabun7= SabunModel(r7_s3, s3=s3, lout=lout)
        x37 = sabun7([x3]) 
        
        sabun8= SabunModel(r8_s3, s3=s3, lout=lout)
        x38 = sabun8([x3]) 
        
        sabun9= SabunModel(r9_s3, s3=s3, lout=lout)
        x39 = sabun9([x3]) 
        
        sabun10= SabunModel(r10_s3, s3=s3, lout=lout)
        x310 = sabun10([x3]) 
        
        sabun11= SabunModel(r11_s3, s3=s3, lout=lout)
        x311 = sabun11([x3]) 
        
        # concatenate all eleven difference
        x3 = Lambda(lambda x: K.concatenate([x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9],x[10]], axis=-1),\
                 output_shape=(1,lout*11))([x31,x32,x33,x34,x35,x36,x37,x38,x39,x310,x311]) # (1,1408)
    
    # flatten to Dense input
    x = Flatten()(x3) # (1408)
    #  transform  (1408) to 10 digits probability using one fully-connected layer
    x = Dense(10, activation='softmax')(x)  # softmax
    
    
    # sequential model definition 
    model = Model(inputs=inp, outputs=x)
    print(model.summary())  # show model summary 
    
    # model compile
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    
    # set tensorboard as callback.  output will be save in log_dir
    tbCallBack = TensorBoard(log_dir='logs', histogram_freq=0, write_graph=True, write_images=True)
    
    
    # weight file
    F_name='mnist_model1_weights.h5'  # file name
    if True:   # load weight if previous weight file exists
        if os.path.exists(F_name):
            model.load_weights(F_name)
            print ('load weights from ', F_name)
    
    
    # set number of epoch !
    N_epoch=100
    # start to train
    model.fit(X_train, Y_train,     # train dataset
              batch_size=128,       # batch size
              epochs= N_epoch,      # epoch
              verbose=1,            # show progress
              validation_data=(X_test, Y_test),  # validation dataset
              callbacks=[tbCallBack]  # callback
             )
    
    # save weight, at last
    model.save_weights( F_name )   

In [ ]:
# show incorrect prediction image with binarization
if True:   
    # set maximum number how many show incorrect prediction image from the beginning
    show_max_number=20
    
    # compute prediction at once
    predict = model.predict(X_test, batch_size=32,verbose=1) # predict of validation data
    
    indexes = np.argmax(predict, axis=1) # get predict label index
    incorrect_indexes=np.arange(len(y_test))[indexes != y_test]  # get incorrect indexes
    print ('number of incorrect image', len(incorrect_indexes))
    
    predict_value= np.amax(predict,axis=1)   # get predict value of predict label
    true_value=np.choose(y_test, predict.T)  # get predict value of true label
    
    for i, id in enumerate(incorrect_indexes): 
        print ('id ', id)
        print ('true_label', y_test[id], ' predict_label', indexes[id])
        print ('value_of_true_label', true_value[id], ' value_of_predict_label', predict_value[id])
        
        # show incorrect digit binarization image
        for y in range(28):
            for x in range(28):
                if  X_test[id,y,x] > 0.0:
                    print ( '1', end='')
                else:
                    print ( '0', end='')
            print('')
        
        if i >= show_max_number :
            break # show stop
    